# Что будет
- сводные таблицы
- фильтры и вычисления с помощью метода loc
- фильтрация пустых значений через isnull
- время в pandas
- строковые методы
- немного про учет форм слов

# Сводные таблицы
Прям как в экселе

In [4]:
import pandas as pd

In [5]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


### Упражнение
Какие оценки поставил пользователь №1 и в каком количестве?

Что делаем:
1. Фильтруем датафрейм ratings для userId = 1
2. Считаем для этого пользователя сколько он выставил единиц, двоек итд

In [6]:
ratings[ratings['userId'] == 1].groupby('rating').agg({'rating': 'count'})

,rating
rating,
1.0,2
2.0,7
2.5,3
3.0,4
3.5,1
4.0,3


Как это сделать для всех пользователей сразу:

In [7]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0).head()


rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0
2,0,2,0,4,0,36,0,23,0,11
3,0,0,0,1,3,18,9,11,4,5
4,0,5,0,5,0,23,0,52,0,119
5,0,0,1,0,3,3,27,42,19,5


In [8]:
# можно итоги добавить
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
userId,,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0,20
2,0,2,0,4,0,36,0,23,0,11,76
3,0,0,0,1,3,18,9,11,4,5,51
4,0,5,0,5,0,23,0,52,0,119,204
5,0,0,1,0,3,3,27,42,19,5,100


### Упражнение
Какой пользователь выставил больше всех пятерок?

In [9]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    )[[5.0]].sort_values(5.0, ascending = False).head()

rating,5.0
userId,
564,408
232,243
242,219
547,214
30,196


### Фильтры и вычисления с помощью loc и iloc

In [37]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


### Но сначала упражнение
Сколько различных страниц есть в столбце url? Постройте топ-5 страниц по посещаемости.

Бонусный вариант: выведите топ-5 страниц без указания хоста https://host.ru

In [38]:
from urllib import parse

urls = log.groupby('url').agg({'visit_id':'count'})
urls['path'] = parse.urlsplit('https://host.ru/05e41944dd85a900a6	').path
urls.head()

,visit_id,path
url,,
https://host.ru/05e41944dd85a900a6,485,/05e41944dd85a900a6\t
https://host.ru/060d9bdbcba66b8bb3,497,/05e41944dd85a900a6\t
https://host.ru/108ce4b365afb7b88e,534,/05e41944dd85a900a6\t
https://host.ru/186c4d6b47dbde0e74,459,/05e41944dd85a900a6\t
https://host.ru/1a0fe145099b114845,497,/05e41944dd85a900a6\t


In [39]:
# метод loc позволяет выбрать строки и столбцы в соответствии с условиями
# двоеточие означает выбор всех значений

log.loc[:, ['user_id', 'region']].head()

,user_id,region
0,b1613cc09f,Russia
1,4c3ec14bee,Russia
2,a8c40697fb,Russia
3,521ac1d6a0,Russia
4,d7323c571c,Russia


In [40]:
# iloc работает аналогично, но с указанием номера строки / столбца

log.iloc[:, [1, 2]].head()

,visit_id,url
0,e3b0c44298,https://host.ru/3c19b4ef7371864fa3
1,6e340b9cff,https://host.ru/c8d9213a31839f9a3a
2,96a296d224,https://host.ru/b8b58337d272ee7b15
3,709e80c884,https://host.ru/b8b58337d272ee7b15
4,df3f619804,https://host.ru/b8b58337d272ee7b15


In [41]:
# пример фильтра на страну
# если столбцы не надо фильтровать, то второй параметр можно не указывать

log.loc[log.region == 'Russia'].head(10)

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email
12,1549980789,15ec7bf0b5,https://host.ru/8e88d4703848cc0ec4,Russia,6d1d81f7f8,paid


In [42]:
# пример вычисления нового столбца с НДС для страны

log.loc[log['region'] == 'Russia', 'VAT'] = 1.2
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,VAT
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,1.2
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,1.2
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,1.2
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,1.2


In [43]:
# вариант с вычисляемым фильтром

log.loc[lambda row: row.region == 'Russia'].head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,VAT
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,1.2
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,1.2
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,1.2
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,1.2
12,1549980789,15ec7bf0b5,https://host.ru/8e88d4703848cc0ec4,Russia,6d1d81f7f8,paid,1.2


### Упражнение
Какие варианты источников трафика есть в столбце traffic_source?

Создайте столбец traffic_type, в котором для источников 'yandex' и 'google' будет стоять значение 'organic'. А для остальных вариантов - NaN.

In [44]:
log.loc[log['traffic_source'].isin(['yandex','google']), 'traffic_type'] = 'Organic'
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,VAT,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2,Organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2,Organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2,Organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2,Organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,1.2,Organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,1.2,NaN
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,1.2,NaN
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,NaN,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,1.2,NaN


### Методы isnull, isna
Определение пустых или None значений. По сути одинаковые методы

In [31]:
import numpy as np

In [32]:
df = pd.DataFrame({'value': [123, None, np.nan, np.NaN, np.NAN, 456]})
df

,value
0,123.0
1,NaN
2,NaN
3,NaN
4,NaN
5,456.0


In [33]:
# фильтр на пустые значения в столбце value

df.loc[pd.isnull(df.value), :]

,value
1,NaN
2,NaN
3,NaN
4,NaN


### Упражнение
Для пустых значений в столбце traffic_type выставьте значение 'other'

In [45]:
log.loc[pd.isnull(log['traffic_type']), 'traffic_type'] = 'other'
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,VAT,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2,Organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2,other
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2,Organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2,Organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2,Organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,1.2,Organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,1.2,other
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,1.2,other
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,NaN,other
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,1.2,other


### Дата и время в pandas

In [48]:
log['date'] = pd.to_datetime(log['timestamp'], unit='s')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,VAT,traffic_type,date
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2,Organic,2019-02-12 14:11:32
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2,other,2019-02-12 14:11:44
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2,Organic,2019-02-12 14:11:55
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2,Organic,2019-02-12 14:12:05
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2,Organic,2019-02-12 14:12:16


In [47]:
# столбец datetime64[ns] теперь имеет тип даты
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18938 entries, 0 to 18937
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   timestamp       18938 non-null  int64         
 1   visit_id        18938 non-null  object        
 2   url             18938 non-null  object        
 3   region          18938 non-null  object        
 4   user_id         18938 non-null  object        
 5   traffic_source  18938 non-null  object        
 6   VAT             10913 non-null  float64       
 7   traffic_type    18938 non-null  object        
 8   date            18938 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 1.3+ MB


In [49]:
# получим час визита

log['hour'] = log.date.dt.hour
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,VAT,traffic_type,date,hour
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2,Organic,2019-02-12 14:11:32,14
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2,other,2019-02-12 14:11:44,14
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2,Organic,2019-02-12 14:11:55,14
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2,Organic,2019-02-12 14:12:05,14
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2,Organic,2019-02-12 14:12:16,14


### Упражнение
Дан датафрейм df с временем визита и соответствующей покупки. Сделайте следующее:
1. Создайте столбцы visit_dt и purchase_dt типа datetime из столбцов visit_datetime и purchase_datetime
2. Посчитайте разницу между временем визита и покупки (простым вычитанием)
3. Переведите полученный столбец с длительностью покупки в минуты:
```python
(df.purchase_dt - df.visit_dt).astype('timedelta64[m]')
```
4. Посчитайте среднее время покупки в минутах

In [50]:
df = pd.DataFrame({'visit_datetime': ['2019-11-04T00:05:13', '2019-11-04T00:06:19', '2019-11-04T01:35:14'], 
                   'purchase_datetime': ['2019-11-04T00:15:53', '2019-11-04T00:07:59', '2019-11-04T03:15:30']})
df

,visit_datetime,purchase_datetime
0,2019-11-04T00:05:13,2019-11-04T00:15:53
1,2019-11-04T00:06:19,2019-11-04T00:07:59
2,2019-11-04T01:35:14,2019-11-04T03:15:30


In [62]:
df['visit_dt'] = pd.to_datetime(df['visit_datetime'])
df['purchase_dt'] = pd.to_datetime(df['purchase_datetime'])
df['delta'] = (df.purchase_dt - df.visit_dt).astype('timedelta64[m]')

In [63]:
df

,visit_datetime,purchase_datetime,visit_dt,purchase_dt,delta
0,2019-11-04T00:05:13,2019-11-04T00:15:53,2019-11-04 00:05:13,2019-11-04 00:15:53,10.0
1,2019-11-04T00:06:19,2019-11-04T00:07:59,2019-11-04 00:06:19,2019-11-04 00:07:59,1.0
2,2019-11-04T01:35:14,2019-11-04T03:15:30,2019-11-04 01:35:14,2019-11-04 03:15:30,100.0


### Методы работы со строками

In [64]:
stats = pd.read_csv('keywords.csv')
stats.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


Проверка наличия подстроки в строке в питоне:

In [65]:
'охотник' in 'каждый охотник желает знать...'

True

Аналог в pandas:

In [66]:
stats[stats.keyword.str.contains('охотник')].head()

,keyword,shows
3072,сумеречные охотники 2 сезон,71965
3474,сумеречные охотники,66083
3654,белоснежка и охотник 2 фильм 2016,63154
4178,последний охотник на ведьм,57560
6127,последний охотник на ведьм фильм 2015,42213


[Документация](https://www.geeksforgeeks.org/python-pandas-series-str-contains/)

Syntax: Series.str.contains(pat, case=True, flags=0, na=nan, regex=True)

Parameter :
- pat : Character sequence or regular expression.
- case : If True, case sensitive.
- flags : Flags to pass through to the re module, e.g. re.IGNORECASE.
- na : Fill value for missing values.
- regex : If True, assumes the pat is a regular expression.

In [67]:
# поиск одного из нескольких слов

stats[stats.keyword.str.contains('охотник|фильм|2016')].head()

,keyword,shows
20,фильмы 2016,4486635
51,фильмы,2156641
54,фильмы онлайн,2305540
68,смотреть фильмы онлайн,1928484
86,порно фильмы,1458031


### Упражнение
Отфильтруйте датафрейм stats по поисковым запросам, которые содержат строку "погода в" и упоминают один из городов: Москва, Новосибирск, Краснодар.

### replace

In [ ]:
'отпуск начнется завтра'.replace('завтра', 'через месяц')

Аналог в pandas на запросах про сериалы:

In [ ]:
serial = stats[stats.keyword.str.contains('сериалы')]
serial.head()

In [ ]:
serial.keyword.str.replace('сериалы', 'книги').head()

### Как учитывать разное написание слов
Самое простое - методы upper и lower

In [ ]:
serial.keyword.str.upper().head()

In [ ]:
serial.keyword.str.lower().head()

### Что делать если нужно учесть формы написания слов?

In [ ]:
stats[stats.keyword.str.contains('рубл')].head()

Почему нельзя просто оставить str.contains('рубл'):

In [ ]:
non_financial_search = 'рубленая котлетка'

### Библиотека [pymystem](https://pypi.org/project/pymystem3/)

In [56]:
from pymystem3 import Mystem

In [57]:
search = 'курс гривны к рублю рубли рублях'

In [58]:
m = Mystem()
lemmas = m.lemmatize(search)
lemmas

['курс',
 ' ',
 'гривна',
 ' ',
 'к',
 ' ',
 'рубль',
 ' ',
 'рубль',
 ' ',
 'рубль',
 '\n']

In [59]:
' '.join(lemmas)

'курс   гривна   к   рубль   рубль   рубль \n'

# Домашнее задание



### Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений


In [96]:
import pandas as pd

log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [97]:
log.loc[log['traffic_source'].isin(['yandex', 'google']), 'source_type'] = 'organic'
log.loc[(log['traffic_source'].str.lower().isin(['paid', 'email'])) & (log['region'].str.lower() == 'russia'), 'source_type'] = 'ad'
log.loc[(log['traffic_source'].str.lower().isin(['paid', 'email'])) & (log['region'].str.lower() != 'russia'), 'source_type'] = 'other'
log.loc[pd.isnull(log['source_type']), 'source_type'] = log['traffic_source'].str.lower()
log

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
...,...,...,...,...,...,...,...
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,other
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,ad
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,ad


### Задание 2
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [98]:
import pandas as pd

log = pd.read_csv('URLs.txt', sep=';')
log.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [99]:
log['isnews'] = log['url'].str.contains('\/\d\d\d\d\d\d\d\d\-',case=False, flags=0,na='NaN',regex=True)
log.loc[log['isnews'] == True].head()

,url,isnews
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...,True
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...,True
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...,True
6,/video/36001498-poyavilis-pervye-podrobnosti-g...,True
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...,True


### Задание 3
В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [100]:
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})
data

,keyword,shows
0,курс гривны к рублю,125076
1,доллары в рубли,114173
2,100 долларов в рублях,97534
3,курс рубля,53546


In [101]:
from pymystem3 import Mystem
data['lemmas'] = data['keyword'].apply(lambda keyword: Mystem().lemmatize(keyword))
data

,keyword,shows,lemmas
0,курс гривны к рублю,125076,"[курс, , гривна, , к, , рубль, \n]"
1,доллары в рубли,114173,"[доллар, , в, , рубль, \n]"
2,100 долларов в рублях,97534,"[100, , доллар, , в, , рубль, \n]"
3,курс рубля,53546,"[курс, , рубль, \n]"
